# 네이버 부동산 데이터 크롤링
http://land.naver.com 데이터 크롤링

https://land.naver.com/article/articleList.nhn?rletTypeCd=A01&tradeTypeCd=all&hscpTypeCd=A01:A03:A04&cortarNo=1168010600

- rletTypeCd: A01=아파트, A02=오피스텔, B01=분양권, 주택=C03, 토지=E03, 원룸=C01, 상가=D02, 사무실=D01, 공장=E02, 재개발=F01, 건물=D03
- tradeTypeCd (거래종류): all=전체, A1=매매, B1=전세, B2=월세, B3=단기임대
- hscpTypeCd (매물종류): 아파트=A01, 주상복합=A03, 재건축=A04 (복수 선택 가능)
- cortarNo(법정동코드): (예: 1168010600 서울시, 강남구, 대치동)

## 사이 문자열 검색
정규식을 사용하여 특정 문자열 사이에 있는 문자열을 얻는다.

In [3]:
import re

text = '220/191공급면적220.92㎡전용면적191.06㎡'

공급면적 = re.findall('공급면적(.*?)㎡', text)[0]
전용면적 = re.findall('전용면적(.*?)㎡', text)[0]

print('공급면적: ', 공급면적)
print('전용면적: ', 전용면적)

공급면적:  220.92
전용면적:  191.06


## 특정 페이지 테이블
지역을 검색하는 경우 다음과 같은 형태의 URL가 된다

https://land.naver.com/article/articleList.nhn?rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04&cortarNo=1168010600

이 페이지의 아래 쪽 테이블(table)을 분석하여 데이터를 추출해보자

In [10]:
import re
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import requests

url = 'https://land.naver.com/article/articleList.nhn?rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04&cortarNo=1168010600'   
r = requests.get(url).text
soup = BeautifulSoup(r)
table = soup.find('table', attrs={'class':'sale_list'})
trs = table.tbody.find_all('tr')

In [11]:
trs

[<tr class="evennum _trow_1925323014">
 <td class="sale_type bottomline" rowspan="2">
 <div class="inner pl4" tabindex="0">매매</div>
 </td>
 <td class="sale_type2 bottomline" rowspan="2" tabindex="0"><div class="inner" style="padding:0;">아파트</div></td>
 <td class="bottomline" rowspan="2">
 <div class="inner inner_mark">
 <span class="mark4" title="확인한지 1개월 이내인 매물"><img alt="확인한지 1개월 이내인 매물" height="15" src="https://ssl.pstatic.net/static.land/static/service/20191007/article/blank.gif" style="background:url(https://ssl.pstatic.net/static.land/static/service/20191007/article/articlelist.gif) -100px -130px no-repeat;" width="48"/>19.10.21.</span>
 </div>
 </td>
 <td class="bottomline _thumb_image" loop_id="p1" rowspan="2">
 <div class="inner inner_thmb">
 <div class="thmb" id="imgView_p1">
 <div _iscppcarticlelinkuseatarticletitle="false" atcl_no="1925323014" atcl_rlet_type_cd="A01" class="thmb_area _thumb_gal_img" img_id="/20191021_191/land_naver_1571632385496BEMBw_JPEG/13414c319780a.jpg"

In [21]:
for tr in trs[::2]:
    tds = tr.find_all('td')
    cols = [' '.join(td.text.strip().split()) for td in tds]
    
    if '_thumb_image' not in tds[3]['class']: # 현장확인 날짜와 이미지가 없는 행
        cols.insert(3, '')

    거래 = cols[0]
    종류 = cols[1]
    확인일자 = datetime.strptime(cols[2], '%y.%m.%d.')
    현장확인 = cols[3]
    매물명 = cols[4]
    면적 = cols[5]
    공급면적 = re.findall('공급면적(.*?)㎡', 면적)[0].replace(',', '')
    전용면적 = re.findall('전용면적(.*?)㎡', 면적)[0].replace(',', '')
    공급면적 = float(공급면적)
    전용면적 = float(전용면적)
    층 = cols[6]
    매물가 = int(cols[7].replace(',', '')) 
    연락처 = cols[8]
        
    print(거래, 종류, 확인일자, 현장확인, 매물명, 공급면적, 전용면적, 층, 매물가, 연락처)

매매 아파트 2019-10-21 00:00:00 0/0 집주인 현대썬앤빌테헤란(도시형) 네이버부동산에서 보기 81.39 48.72 7/7 85000 예일공인중개사 02-508-5533
매매 아파트 2019-10-21 00:00:00  집주인 래미안대치팰리스1단지 네이버부동산에서 보기 125.23 94.49 10/26 320000 신세계공인중개사사무소 02-567-6969
매매 아파트 2019-10-21 00:00:00  집주인 래미안대치팰리스1단지 네이버부동산에서 보기 112.24 84.98 22/35 288000 신세계공인중개사사무소 02-567-6969
매매 아파트 2019-10-18 00:00:00  집주인 래미안대치팰리스1단지 네이버부동산에서 보기 126.22 94.49 6/22 320000 명성공인중개사 02-3463-3355
매매 아파트 2019-10-22 00:00:00  래미안대치팰리스1단지 네이버부동산에서 보기 113.15 84.97 13/35 275000 청실두꺼비공인중개사사무소 02-564-7500
매매 아파트 2019-10-22 00:00:00  개포우성2차 네이버부동산에서 보기 101.09 94.74 12/15 250000 위드공인중개사무소 02-554-8888
매매 아파트 2019-10-22 00:00:00  선경1,2차 네이버부동산에서 보기 148.6 127.75 11/15 320000 제이스공인 02-556-8245
매매 아파트 2019-10-22 00:00:00  래미안대치팰리스1단지 네이버부동산에서 보기 125.23 94.49 10/25 310000 청실두꺼비공인중개사사무소 02-564-7500
매매 아파트 2019-10-22 00:00:00  래미안대치팰리스1단지 네이버부동산에서 보기 112.24 84.98 18/35 275000 한스공인중개사사무소 02-567-8855
매매 아파트 2019-10-22 00:00:00  래미안대치팰리스1단지 네이버부동산에서 보기 112.24 84.98 4/35 275000 대치수성부동산중개사무

## 함수로 만들기

In [22]:
import re
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup

def get_naver_realasset(area_code):
    url = 'https://land.naver.com/article/articleList.nhn?' \
        + 'rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04' \
        + '&cortarNo=' + area_code

    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    table = soup.find('table')
    trs = table.tbody.find_all('tr')
    
    value_list = []

    # 거래, 종류, 확인일자, 매물명, 매물명, 면적(㎡), 층, 매물가(만원), 연락처
    for tr in trs[::2]:
        tds = tr.find_all('td')
        cols = [' '.join(td.text.strip().split()) for td in tds]

        if '_thumb_image' not in tds[3]['class']: # 현장확인 날짜와 이미지가 없는 행
            cols.insert(3, '')

        거래 = cols[0]
        종류 = cols[1]
        확인일자 = datetime.strptime(cols[2], '%y.%m.%d.')
        현장확인 = cols[3]
        매물명 = cols[4]
        면적 = cols[5]
        공급면적 = re.findall('공급면적(.*?)㎡', 면적)[0].replace(',', '')
        전용면적 = re.findall('전용면적(.*?)㎡', 면적)[0].replace(',', '')
        공급면적 = float(공급면적)
        전용면적 = float(전용면적)
        층 = cols[6]
        매물가 = int(cols[7].replace(',', '')) 
        연락처 = cols[8]
        
        value_list.append([거래, 종류, 확인일자, 현장확인, 매물명, 공급면적, 전용면적, 층, 매물가, 연락처])
        
    cols = ['거래', '종류', '확인일자', '현장확인', '매물명', '공급면적', '전용면적', '층', '매물가', '연락처']
    df = pd.DataFrame(value_list, columns=cols)
    return df

In [23]:
df = get_naver_realasset('1168010600')
df[:20]

,거래,종류,확인일자,현장확인,매물명,공급면적,전용면적,층,매물가,연락처
0,매매,아파트,2019-10-21,0/0,집주인 현대썬앤빌테헤란(도시형) 네이버부동산에서 보기,81.39,48.72,7/7,85000,예일공인중개사 02-508-5533
1,매매,아파트,2019-10-21,,집주인 래미안대치팰리스1단지 네이버부동산에서 보기,112.24,84.98,22/35,288000,신세계공인중개사사무소 02-567-6969
2,매매,아파트,2019-10-21,,집주인 래미안대치팰리스1단지 네이버부동산에서 보기,125.23,94.49,10/26,320000,신세계공인중개사사무소 02-567-6969
3,매매,아파트,2019-10-18,,집주인 래미안대치팰리스1단지 네이버부동산에서 보기,126.22,94.49,6/22,320000,명성공인중개사 02-3463-3355
4,매매,아파트,2019-10-22,0/0,래미안대치팰리스1단지 네이버부동산에서 보기,125.23,94.49,14/25,315000,한스공인중개사사무소 02-567-8855
5,매매,아파트,2019-10-22,,대치아이파크 네이버부동산에서 보기,179.41,149.78,3/25,320000,대치박사공인중개사사무소 02-554-5999
6,매매,아파트,2019-10-22,,개포우성2차 네이버부동산에서 보기,101.09,94.74,12/15,250000,위드공인중개사무소 02-554-8888
7,매매,아파트,2019-10-22,0/0,래미안대치팰리스1단지 네이버부동산에서 보기,113.15,84.97,20/30,275000,한스공인중개사사무소 02-567-8855
8,매매,아파트,2019-10-22,,개포우성1차 네이버부동산에서 보기,148.76,127.61,5/15,320000,위드공인중개사무소 02-554-8888
9,매매,아파트,2019-10-22,,래미안대치팰리스2단지 네이버부동산에서 보기,123.48,91.89,16/35,275000,청실두꺼비공인중개사사무소 02-564-7500


In [31]:
import re
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup

def get_naver_realasset(area_code, page=1): # default page to changable
    url = 'https://land.naver.com/article/articleList.nhn?' \
        + 'rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04' \
        + '&cortarNo=' + area_code \
        + '&page=' + str(page)

    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    table = soup.find('table')
    trs = table.tbody.find_all('tr')
    if '등록된 매물이 없습니다' in trs[0].text:
        return pd.DataFrame()

    value_list = []

    # 거래, 종류, 확인일자, 매물명, 면적(㎡), 층, 매물가(만원), 연락처
    for tr in trs[::2]:
        tds = tr.find_all('td')
        cols = [' '.join(td.text.strip().split()) for td in tds]

        if '_thumb_image' not in tds[3]['class']: # 현장확인 날짜와 이미지가 없는 행
            cols.insert(3, '')

        # print(cols)
        거래 = cols[0]
        종류 = cols[1]
        확인일자 = datetime.strptime(cols[2], '%y.%m.%d.')
        현장확인 = cols[3]
        매물명 = cols[4]
        면적 = cols[5]
        공급면적 = re.findall('공급면적(.*?)㎡', 면적)[0].replace(',', '')
        전용면적 = re.findall('전용면적(.*?)㎡', 면적)[0].replace(',', '')
        공급면적 = float(공급면적)
        전용면적 = float(전용면적)
        층 = cols[6]
        if cols[7].find('호가일뿐 실거래가로확인된 금액이 아닙니다') >= 0:
            pass # 단순호가 별도 처리하고자 하면 내용 추가
        매물가 = int(cols[7].split(' ')[0].replace(',', '')) 
        연락처 = cols[8]

        value_list.append([거래, 종류, 확인일자, 현장확인, 매물명, 공급면적, 전용면적, 층, 매물가, 연락처])
        
    cols = ['거래', '종류', '확인일자', '현장확인', '매물명', '공급면적', '전용면적', '층', '매물가', '연락처']
    df = pd.DataFrame(value_list, columns=cols)
    return df

In [32]:
df = get_naver_realasset('1168010600', 3) # 3 페이지
df

,거래,종류,확인일자,현장확인,매물명,공급면적,전용면적,층,매물가,연락처
0,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,112.24,84.98,29/35,280000,래미안현대공인중개사사무소 02-566-3993
1,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,112.24,84.98,16/21,276000,래미안대치팰리스공인중개사 02-563-0080
2,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,125.23,94.49,11/26,310000,래미안대치팰리스공인중개사 02-563-0080
3,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,125.23,94.49,15/25,320000,래미안대치팰리스공인중개사 02-563-0080
4,매매,아파트,2019-10-22,0/0,래미안대치팰리스1단지 네이버부동산에서 보기,126.22,94.49,14/26,310000,래미안대치팰리스공인중개사 02-563-0080
5,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,126.22,94.49,7/25,320000,대치박사공인중개사사무소 02-554-5999
6,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,112.24,84.98,14/35,273000,한스공인중개사사무소 02-567-8855
7,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,113.15,84.97,12/18,275000,래미안대치팰리스공인중개사 02-563-0080
8,매매,아파트,2019-10-22,,개포우성2차 네이버부동산에서 보기,180.54,169.18,8/15,400000,아성공인중개사사무소 02-555-9918
9,매매,아파트,2019-10-22,,"선경1,2차 네이버부동산에서 보기",182.22,160.76,6/15,370000,아성공인중개사사무소 02-555-9918


## 페이지 묶어 DataFrame 생성하기
페이지별로 생성된 DataFrame 을 묶어(append) 하나의 DataFrame으로 합친다

In [33]:
area_code = '1168010600' # 강남구, 대치동 

df = pd.DataFrame()
for i in range(1, 20):
    print(i, end=', ')
    df_tmp = get_naver_realasset(area_code, i)
    if len(df_tmp) <= 0: # 더이상 데이터가 없는 페이지
        break
    df = df.append(df_tmp, ignore_index=True)
df.head(10)

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 

,거래,종류,확인일자,현장확인,매물명,공급면적,전용면적,층,매물가,연락처
0,매매,아파트,2019-10-21,0/0,집주인 현대썬앤빌테헤란(도시형) 네이버부동산에서 보기,81.39,48.72,7/7,85000,예일공인중개사 02-508-5533
1,매매,아파트,2019-10-21,,집주인 래미안대치팰리스1단지 네이버부동산에서 보기,112.24,84.98,22/35,288000,신세계공인중개사사무소 02-567-6969
2,매매,아파트,2019-10-21,,집주인 래미안대치팰리스1단지 네이버부동산에서 보기,125.23,94.49,10/26,320000,신세계공인중개사사무소 02-567-6969
3,매매,아파트,2019-10-18,,집주인 래미안대치팰리스1단지 네이버부동산에서 보기,126.22,94.49,6/22,320000,명성공인중개사 02-3463-3355
4,매매,아파트,2019-10-22,,대치아이파크 네이버부동산에서 보기,179.41,149.78,3/25,320000,대치박사공인중개사사무소 02-554-5999
5,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,125.23,94.49,10/25,310000,청실두꺼비공인중개사사무소 02-564-7500
6,매매,아파트,2019-10-22,,개포우성2차 네이버부동산에서 보기,101.09,94.74,12/15,250000,위드공인중개사무소 02-554-8888
7,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,112.24,84.98,18/35,275000,한스공인중개사사무소 02-567-8855
8,매매,아파트,2019-10-22,,"선경1,2차 네이버부동산에서 보기",148.60,127.75,11/15,320000,제이스공인 02-556-8245
9,매매,아파트,2019-10-22,,개포우성1차 네이버부동산에서 보기,148.76,127.61,13/15,320000,위드공인중개사무소 02-554-8888


In [34]:
df.shape

(570, 10)

In [35]:
df

,거래,종류,확인일자,현장확인,매물명,공급면적,전용면적,층,매물가,연락처
0,매매,아파트,2019-10-21,0/0,집주인 현대썬앤빌테헤란(도시형) 네이버부동산에서 보기,81.39,48.72,7/7,85000,예일공인중개사 02-508-5533
1,매매,아파트,2019-10-21,,집주인 래미안대치팰리스1단지 네이버부동산에서 보기,112.24,84.98,22/35,288000,신세계공인중개사사무소 02-567-6969
2,매매,아파트,2019-10-21,,집주인 래미안대치팰리스1단지 네이버부동산에서 보기,125.23,94.49,10/26,320000,신세계공인중개사사무소 02-567-6969
3,매매,아파트,2019-10-18,,집주인 래미안대치팰리스1단지 네이버부동산에서 보기,126.22,94.49,6/22,320000,명성공인중개사 02-3463-3355
4,매매,아파트,2019-10-22,,대치아이파크 네이버부동산에서 보기,179.41,149.78,3/25,320000,대치박사공인중개사사무소 02-554-5999
5,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,125.23,94.49,10/25,310000,청실두꺼비공인중개사사무소 02-564-7500
6,매매,아파트,2019-10-22,,개포우성2차 네이버부동산에서 보기,101.09,94.74,12/15,250000,위드공인중개사무소 02-554-8888
7,매매,아파트,2019-10-22,,래미안대치팰리스1단지 네이버부동산에서 보기,112.24,84.98,18/35,275000,한스공인중개사사무소 02-567-8855
8,매매,아파트,2019-10-22,,"선경1,2차 네이버부동산에서 보기",148.60,127.75,11/15,320000,제이스공인 02-556-8245
9,매매,아파트,2019-10-22,,개포우성1차 네이버부동산에서 보기,148.76,127.61,13/15,320000,위드공인중개사무소 02-554-8888
